In [0]:
#       .
#       |
#       |
#    ,-'"`-.
#  ,'       `.
#  |  _____  |      .-( HEY baby,lets go out)
#  | (_o_o_) |    ,'    ( and kill all humans.)
#  |         | ,-'
#  | |HHHHH| |
#  | |HHHHH| |
#-'`-._____.-'`-

#Utilizando uma Rede Neural para predição de fala do seriado Friends

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import torch
import string
import random
import re
import time, math

import torch.utils.data as utils
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import seaborn as sn
from sklearn.metrics import confusion_matrix

import torchvision
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import collections

import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.utils.data as utils
import csv


def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [0]:
from google.colab import files

with open('train_dataset.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    #line_count = 0
   # for row in csv_reader:
       # print(f'\t Frase {row[0]}: {row[1]}  Label/personagem: {row[2]}.')  

#Treinamento

In [0]:
# carregar o dataset de treino

raw_data1 = pd.read_csv('train_dataset.txt')
raw_data1.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [0]:
# remover colunas não relevantes

data1 = raw_data1.drop(["Sr No.", "Emotion", "Sentiment","Dialogue_ID","Utterance_ID","Season","Episode","StartTime","EndTime"], axis= 1)
data1.head()

,Utterance,Speaker
0,also I was the point person on my companys tr...,Chandler
1,You mustve had your hands full.,The Interviewer
2,That I did. That I did.,Chandler
3,So lets talk a little bit about your duties.,The Interviewer
4,My duties? All right.,Chandler


In [0]:
print("The number of rows in train_dataset are {}".format(len(data1)))

The number of rows in train_dataset are 9989


In [0]:
#Normalizar o dataset com targets com 1 char apenas
# sendo eles: 
#   c= Chandler
#   r = Rachel
#   R = Ross
#   m = Monica
#   p = Phoebe
#   j = Joey
#   o = others speakers


for i in range(len(data1)):
    if(data1.iloc[i][-1] == 'Chandler'):
        data1.loc[i][-1] = 'c'
    elif(data1.iloc[i][-1] == 'Rachel'):
        data1.loc[i][-1] = 'r'
    elif(data1.iloc[i][-1] == 'Ross'):
        data1.loc[i][-1] = 'R'
    elif(data1.iloc[i][-1] == 'Monica'):
        data1.loc[i][-1] = 'm'
    elif(data1.iloc[i][-1] == 'Phoebe'):
        data1.loc[i][-1] = 'p'
    elif(data1.iloc[i][-1] == 'Joey'):
        data1.loc[i][-1] = 'j'
    else:
        data1.loc[i][-1] = 'o'
    
data1.sample(frac=1) 
data1.head()

,Utterance,Speaker
0,also I was the point person on my companys tr...,c
1,You mustve had your hands full.,o
2,That I did. That I did.,c
3,So lets talk a little bit about your duties.,o
4,My duties? All right.,c


In [0]:
#normalizar o dataset, tirando os diálogos com menos de 20 de legth
 #if(len(data1.iloc[i][0])<20):
       # data1 = data1.drop([i], axis=0)

#data1 = data1.drop([0], axis=0)
#data1 = data1.drop([1], axis=0)
data1 = data1[data1['Utterance'].map(len) > 20]
data1 = data1[data1['Speaker'] != 'o']

In [0]:
print(len(data1))
data1.head()

5568


,Utterance,Speaker
0,also I was the point person on my companys tr...,c
2,That I did. That I did.,c
4,My duties? All right.,c
10,No dont I beg of you!,c
14,But then who? The waitress I went out with las...,j


In [0]:
#np.savetxt(r'C:\Users\mo0n\Documents\Faculdadestuff\Rede Neural\test.txt', data1.values, fmt='%s')

In [0]:
file = open('test.txt').read()
all_characters = ''.join(set(file))
n_characters = len(all_characters)
print(all_characters)
print(n_characters)
file_len = len(file)
print('file_len =', file_len)


T"u9dN(8tX…7&'Hóqp6Abx$BeJhyK—#*Z:si aG?onSwWO!QvFr5LUYf31gcl04kRm.M,IzP2é-CD/j][;V’E%
94
file_len = 483211


In [0]:
chunk_len = 150

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    index = random.randint(0, len(data1)-1)
    
    data = data1.iloc[index][0]
    
    for i in range(200-len(data)):#normaliza os chunks com tamanho igual
        data+='/0'
        
    return data, index
    
print(random_chunk())

('Wh\x97no, but y\x92know who did stop in here looking for ya, Tennille./0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0', 1647)


In [0]:

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([40, 22, 65, 84, 92, 52])


In [0]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk, index = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = data1.iloc[index][-1]
    target = char_tensor(target)
    return inp, target

print(random_training_set())
#tensor = random_training_set()
#print(tensor)
#if(tensor[0][199] == 3) :
   # print("sim!")

(tensor([ 2, 29, 40, 10, 15, 37, 39,  6, 38, 58, 58, 27, 53, 27, 44, 10, 49, 39,
        76, 39, 46, 40, 37, 39,  6, 53,  4, 44, 69, 39, 40, 44,  6, 39, 37, 10,
         4, 19, 38,  6, 49, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85, 67, 85,
        67, 85, 67, 85, 67, 85, 67, 85,

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, hidden2):
        input = self.encoder(input.view(1, -1))
        output, (hidden, hidden2) = self.lstm(input.view(1, 1, -1), (hidden, hidden2))
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden, hidden2

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=1, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    hidden2 = rnn.init_hidden().cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = ''

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, hidden2 = rnn(prime_input[p], hidden, hidden2)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, hidden2 = rnn(inp, hidden, hidden2)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [0]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    hidden2 = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    hidden2 = hidden2.cuda()
    for c in range(chunk_len):
        if(inp[c] != 27):
            output, hidden, hidden2 = rnn(inp[c], hidden, hidden2)
            loss += loss_metric(output, target[0].unsqueeze(0))
        else:
            break

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
n_epochs = 30000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    #print(example[0])
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
       # print(evaluate('The', 100), '\n')


 4s (100 0%) 0.2843]
 9s (200 0%) 0.1098]
 14s (300 1%) 0.0777]
 17s (400 1%) 0.1505]
 24s (500 1%) 0.3692]
 29s (600 2%) 0.2480]
 35s (700 2%) 0.1512]
 38s (800 2%) 0.0257]
 42s (900 3%) 0.1125]
 47s (1000 3%) 0.0697]
 51s (1100 3%) 0.0254]
 56s (1200 4%) 0.2713]
 3s (1300 4%) 0.1767]
 6s (1400 4%) 0.0175]
 10s (1500 5%) 0.0457]
 14s (1600 5%) 0.3350]
 19s (1700 5%) 0.3319]
 23s (1800 6%) 0.0776]
 28s (1900 6%) 1.8097]
 31s (2000 6%) 0.0135]
 36s (2100 7%) 0.0130]
 40s (2200 7%) 0.0610]
 43s (2300 7%) 0.1636]
 49s (2400 8%) 0.0075]
 53s (2500 8%) 0.4687]
 58s (2600 8%) 0.1432]
 2s (2700 9%) 0.0130]
 7s (2800 9%) 0.2376]
 11s (2900 9%) 0.0551]
 16s (3000 10%) 0.0522]
 21s (3100 10%) 0.2523]
 25s (3200 10%) 0.1236]
 30s (3300 11%) 0.0095]
 35s (3400 11%) 0.0118]
 39s (3500 11%) 0.1639]
 44s (3600 12%) 0.4200]
 48s (3700 12%) 0.4032]
 53s (3800 12%) 0.2653]
 57s (3900 13%) 0.0855]
 2s (4000 13%) 0.0579]
 9s (4100 13%) 0.1527]
 15s (4200 14%) 0.2168]
 18s (4300 14%) 0.0877]
 23s (4400 14%

In [0]:
print(evaluate("Oh, how bad is it?", 1), '\n')

r 



#Test

In [0]:
# carregar o dataset de test

raw_data2 = pd.read_csv('test_dataset.txt')
raw_data2.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [0]:
# remover colunas não relevantes

data2 = raw_data2.drop(["Sr No.", "Emotion", "Sentiment","Dialogue_ID","Utterance_ID","Season","Episode","StartTime","EndTime"], axis= 1)
data2.head()

,Utterance,Speaker
0,Why do all youre coffee mugs have numbers on ...,Mark
1,Oh. Thats so Monica can keep track. That way ...,Rachel
2,Y'know what?,Rachel
3,"Come on, Lydia, you can do it.",Joey
4,Push!,Joey


In [0]:
#Normalizar o dataset com targets com 1 char apenas
# sendo eles: 
#   c= Chandler
#   r = Rachel
#   R = Ross
#   m = Monica
#   p = Phoebe
#   j = Joey
#   o = others speakers


for i in range(len(data2)):
    if(data2.iloc[i][-1] == 'Chandler'):
        data2.loc[i][-1] = 'c'
    elif(data2.iloc[i][-1] == 'Rachel'):
        data2.loc[i][-1] = 'r'
    elif(data2.iloc[i][-1] == 'Ross'):
        data2.loc[i][-1] = 'R'
    elif(data2.iloc[i][-1] == 'Monica'):
        data2.loc[i][-1] = 'm'
    elif(data2.iloc[i][-1] == 'Phoebe'):
        data2.loc[i][-1] = 'p'
    elif(data2.iloc[i][-1] == 'Joey'):
        data2.loc[i][-1] = 'j'
    else:
        data2.loc[i][-1] = 'o'

        
data2.sample(frac=1)        
        
data2.head()


,Utterance,Speaker
0,Why do all youre coffee mugs have numbers on ...,o
1,Oh. Thats so Monica can keep track. That way ...,r
2,Y'know what?,r
3,"Come on, Lydia, you can do it.",j
4,Push!,j


In [0]:
#normalizar o dataset, tirando os diálogos com menos de 20 de legth
 #if(len(data1.iloc[i][0])<20):
       # data1 = data1.drop([i], axis=0)

#data1 = data1.drop([0], axis=0)
#data1 = data1.drop([1], axis=0)
data2 = data2[data2['Utterance'].map(len) > 20]
data2 = data2[data2['Speaker'] != 'o']

In [0]:
print(len(data2))
data2.head()

1468


,Utterance,Speaker
1,Oh. Thats so Monica can keep track. That way ...,r
3,"Come on, Lydia, you can do it.",j
5,"Push 'em out, push 'em out, harder, harder.",j
6,"Push 'em out, push 'em out, way out!",j
7,"Let's get that ball and really move, hey, hey,...",j


In [0]:
#usando o evaluate para avaliar o modelo com o dataset de test
correct = 0
loss = 0

for i in range(len(data2)):
    predict = evaluate(data2.iloc[i][0])
    label = data2.iloc[i][-1]
    if(predict == label):
        correct+=1
    
        
        
        
accuracy = correct/(len(data2))



print('Acurácia: %d%%'  % (accuracy * 100))

Acurácia: 20%
